In [1]:
# 런타임 초기화 후 제일 먼저
import gc
gc.collect()

123

In [2]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import reduce

In [3]:
train_6 = pd.read_csv('train_6.채널정보_98이상제거.csv')
segment = pd.read_csv("Segment_병합.csv")

In [5]:
# 1️⃣ concat으로 segment 컬럼 붙이기
train_full = pd.concat([train_6, segment['Segment']], axis=1)

In [6]:
train_full

,기준년월,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,이용메뉴건수_ARS_B0M,방문횟수_PC_R6M,방문일수_PC_R6M,...,IB문의건수_CL_RV_R6M,불만제기후경과월_R12M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M,Segment
0,201807,TRAIN_000000,10회 이상,10회 이상,8,6,0,6,1회 이상,1회 이상,...,0,12,22,221,6,0,0,0,0,D
1,201807,TRAIN_000001,1회 이상,1회 이상,0,0,0,0,1회 이상,1회 이상,...,0,12,0,0,0,0,0,0,0,E
2,201807,TRAIN_000002,1회 이상,1회 이상,1,1,0,5,10회 이상,10회 이상,...,0,12,0,0,0,11,6,5,5,C
3,201807,TRAIN_000003,10회 이상,10회 이상,10,6,0,6,1회 이상,1회 이상,...,0,12,23,219,6,0,0,0,0,D
4,201807,TRAIN_000004,1회 이상,1회 이상,0,0,0,0,1회 이상,1회 이상,...,0,0,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,1회 이상,1회 이상,0,0,0,0,1회 이상,1회 이상,...,0,0,0,0,0,0,0,0,0,E
2399996,201812,TRAIN_399996,1회 이상,1회 이상,0,0,0,0,1회 이상,1회 이상,...,0,12,0,0,1,0,0,0,0,D
2399997,201812,TRAIN_399997,1회 이상,1회 이상,0,0,0,0,1회 이상,1회 이상,...,0,12,0,0,0,0,0,0,0,C
2399998,201812,TRAIN_399998,1회 이상,1회 이상,0,0,0,0,1회 이상,1회 이상,...,0,0,0,0,0,0,0,0,0,E


In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [8]:
target = train_full['Segment']
X = train_full.drop(columns=['ID', 'Segment'])

In [9]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [10]:
from sklearn.preprocessing import LabelEncoder

# 문자열 target → 숫자형으로 변환
le = LabelEncoder()
target_encoded = le.fit_transform(target)

print("\n✅ 타겟 라벨 인코딩 완료")
print(pd.Series(target_encoded).value_counts())


✅ 타겟 라벨 인코딩 완료
4    1922052
3     349242
2     127590
0        972
1        144
Name: count, dtype: int64


In [11]:
corrs = X_encoded.corrwith(pd.Series(target_encoded))
corrs_sorted = corrs.abs().sort_values(ascending=False)

print("\n✅ 상관계수 (절대값 내림차순):")
print(corrs_sorted)


✅ 상관계수 (절대값 내림차순):
불만제기후경과월_R12M         1.738991e-01
방문후경과월_PC_R6M         1.727725e-01
홈페이지_금융건수_R6M         1.692645e-01
방문월수_PC_R6M           1.605200e-01
홈페이지_금융건수_R3M         1.602799e-01
                          ...     
방문횟수_PC_R6M_40회 이상    9.254927e-03
IB문의건수_결제일변경_R6M      6.406865e-03
IB문의건수_비밀번호_R6M       3.630396e-03
IB문의건수_CL_RV_R6M      5.031004e-04
기준년월                  1.939854e-16
Length: 63, dtype: float64


In [12]:
corrs_sorted = corrs.sort_values(ascending=False)
print(corrs_sorted.round(3))

방문후경과월_PC_R6M        0.173
방문후경과월_앱_R6M         0.158
방문횟수_PC_R6M_1회 이상    0.148
방문일수_PC_R6M_1회 이상    0.142
방문횟수_앱_R6M_1회 이상     0.137
                     ...  
인입일수_ARS_R6M        -0.157
홈페이지_금융건수_R3M       -0.160
방문월수_PC_R6M         -0.161
홈페이지_금융건수_R6M       -0.169
불만제기후경과월_R12M       -0.174
Length: 63, dtype: float64


In [13]:
corrs_sorted = corrs.reindex(corrs.abs().sort_values(ascending=False).index)
print(corrs_sorted.round(3))


불만제기후경과월_R12M        -0.174
방문후경과월_PC_R6M         0.173
홈페이지_금융건수_R6M        -0.169
방문월수_PC_R6M          -0.161
홈페이지_금융건수_R3M        -0.160
                      ...  
방문횟수_PC_R6M_40회 이상   -0.009
IB문의건수_결제일변경_R6M      0.006
IB문의건수_비밀번호_R6M       0.004
IB문의건수_CL_RV_R6M      0.001
기준년월                 -0.000
Length: 63, dtype: float64
